In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
#export
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
from google.cloud import bigquery
from am4894bq.schema import get_schema, df_to_bq_schema, schema_diff, update_bq_schema, update_df_schema
from am4894bq.utils import does_table_exist

In [ ]:
# default_exp pd

# pd

> Pandas related BigQuery functionality.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export


def df_to_gbq(df: pd.DataFrame, destination_table: str, project_id: str, if_exists: str = 'append') -> True:
    """
    Save df to BigQuery enforcing schema consistency between df and destination table if it exists.
    """
    
    table_id = f'{project_id}.{destination_table}'
    old_schema = get_schema(table_id)
    new_schema = df_to_bq_schema(df)
    diffs = schema_diff(old_schema, new_schema)
    
    bq_client = bigquery.Client()
    
    if does_table_exist(bq_client, table_id):
        # update the table schema in BigQuery
        update_bq_schema(bq_client, table_id, diffs)

    # update the df schema to be as expected by BigQuery
    df = update_df_schema(bq_client, table_id, diffs, df)
    
    # load to BigQuery
    df.to_gbq(destination_table, project_id=project_id, if_exists=if_exists)
    
    return True

